# ITk component population timeline plot tutorial

In this tutorial we will create a plot of component population against time in the hope of generating a timeline of component stage migration. To do this we will be using some 3rd party software to: handle the time stamping of the data we will pull form the ITk database (influx), plotting the time stamped data (altair) and creating an interactive report of our plot to share (datapane). Before we begin lets us install some dependencies:

In [ ]:
# install dependencies
requirements = ['pandas', 'itkdb', 'datapane', 'altair', 'influxdb_client']
with open('requirements.txt', 'w') as my_list:
    for element in requirements:
        my_list.write('%s\n' % element)
!pip install -r requirements.txt

In [ ]:
### get itkdb for PDB interaction
import os
import sys
#pip install pandas
import pandas as pd
import copy
import json
from datetime import datetime, timedelta
import time
#pip install itkdb
import itkdb
import itkdb.exceptions as itkX
# visualisation
#pip install datapane
#pip install altair
import altair as alt
import datapane as dp
# influx 
#pip install influxdb_client
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS
from influxdb_client.client.exceptions import InfluxDBError
import warnings
from influxdb_client.client.warnings import MissingPivotFunction

warnings.simplefilter("ignore", MissingPivotFunction)

In [ ]:
#set up ITk db
user = itkdb.core.User(accessCode1="", accessCode2="")
user.authenticate()
myClient = itkdb.Client(user=user)
print(user.name+" your token expires in "+str(myClient.user.expires_in)+" seconds")

In [ ]:
### some filters and pull some data
myProjCode="S" #s for strips 
myCompCode="MODULE"
myInstCode="GL"  #potential institue codes include:'IHEP', 'RAL', 'LIV', 'GL', 'BHM', 'SHF', 'CAM', 'OX','QMUL','CERN'
compList = myClient.get('listComponents', json={'project':myProjCode, 'currentLocation':myInstCode})
compList.page_info

In [ ]:
#put data in pandas dataframe 
myData=myClient.get('listComponents', json={'project':"S",'currentLocation':myInstCode})
df_data=pd.json_normalize(myData, sep = "_")
df_data

In [ ]:
#group componentTypes
df_data_grp=df_data.groupby(by=["componentType_code"]).count().reset_index()
df_data_grp

### Influx install 
Before we proceed we will need to install influx and launch the daemon. please follow the instructions [on this page](https://portal.influxdata.com/downloads/) to install influxdb and then open the terminal and launch the daemon by typing "influxd" in the terminal (macOS). You need to acces the influx gui by typing [http://localhost:8086](http://localhost:8086/) in your browser of choice, continue with "Get started" and "Setup initial user".

Input the Initial Organization Name in org_local and Initial Bucket Name in bucketName below. 

In [ ]:
#lets try a quick write to influx
#
org_local = ""
bucketName=""

# Store the URL of your InfluxDB instance
url_local="http://localhost:8086"

After that: hit configure later, click the icon that is an arrow about a flat line, click API tokens, click on your token and copy and pase the token in to the varible below.

In [ ]:
token_local = "Your token goes here"

In [ ]:
### set client to get access to influx
clientV2_local = influxdb_client.InfluxDBClient(
   url=url_local,
   token=token_local,
   org=org_local
)

In [ ]:
### set bucket api
buckets_api_local = clientV2_local.buckets_api()

### list buckets (by name)
try:
    #print([x.name for x in buckets_api_remote.find_buckets().buckets])
    database_list=[x.name for x in clientV2_local.buckets_api().find_buckets().buckets]
    print(database_list)
except:
    print("cannot get buckets")

In [ ]:
### what bucket?
### create bucket
#buckets_api_local.create_bucket(bucket_name=bucketName, org=org_local)

### find bucket
buckets_api_local.find_bucket_by_name(bucketName)

### delete bucket
#buckets_api_local.delete_bucket(buckets_api_local.find_bucket_by_name(bucketName).id)

In [ ]:
### set write api
write_api_local = clientV2_local.write_api(write_options=SYNCHRONOUS)

In [ ]:
#write the data
for i in range(0,len(df_data_grp)):
    def loc(x,y):
        a=df_data_grp.loc[x,y]
        return(a)
    data="Strips_in_"+str(myInstCode)+",componentType_code="+str(loc(i,"componentType_code"))+",id="+str(loc(i,"id"))+" "+"code="+str(loc(i,"code"))
    # write
    write_api_local.write(bucketName, org_local, data)
    print(data)

In [ ]:
#setup query API

query_api_local = clientV2_local.query_api(
)

In [ ]:
#query the bucket

query='''
from(bucket: "Bucketname")
  |> range(start: 0, stop: now())
  |> filter(fn: (r) => r["_measurement"] == "Strips_in_XYZ")
  |> filter(fn: (r) => r["_field"] == "code")
  |> yield(name: "mean")
'''
a=query.replace("Bucketname",str(bucketName))
b=a.replace("XYZ",str(myInstCode))
print(b)

In [ ]:
#display query result

query_result = query_api_local.query_data_frame(org=org_local, query=b)
display(query_result)

In [ ]:
#drop down selector for graph
a=query_result.drop_duplicates(subset='componentType_code')
query_list=[None] + list(a['componentType_code'])
input_dropdown = alt.binding_select(options=query_list,name="Component Type Code:   ")
selection = alt.selection_single(
    fields=["componentType_code"], bind=input_dropdown,
)

In [ ]:
#make plot
alt.data_transformers.disable_max_rows()
chart=alt.Chart(query_result).mark_line(point=True).encode(
    x=alt.X('_time',title='Timeline'),
    y=alt.Y('_value',title='Population'),
    color=alt.Color('componentType_code', legend=alt.Legend(title='Component Code')),
    tooltip=['_time','_value','componentType_code']
).properties(
    title={
      "text": "Component migration at Glasgow", 
      "subtitle": "Strips"
    },
    width=600,
    height=350
).interactive().add_selection(selection).transform_filter(selection)


chart.resolve_scale('independent')

If everything has worked the graph about should be populated by points! You can run this script again and the points will turn into lines! If you keep running this script repeatedly for a while you'll start to changes in the population of components in the various stages. We can now move on to creating a datapane report to share our plot.

For this next step we will have to create a datapane account [here is the link](https://datapane.com/getting-started/#), i would recomend loging with your GitHub account. Once you have done so please copy and paste your API token into the funciton below.

In [ ]:
dp.login(token='')

In [ ]:
dp.Report(
    dp.Plot(chart),
    dp.DataTable(query_result)
).upload(name="Component migration at "+str(myInstCode)+": Strips")

God's speed